In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("C:/Users/akshi/OneDrive/Desktop/DA/project_3/ecommerce_user_segmentation.csv")
df.head(5)


,Customer_ID,Recency,Frequency,Monetary,Avg_Order_Value,Session_Count,Avg_Session_Duration,Pages_Viewed,Clicks,Campaign_Response,Wishlist_Adds,Cart_Abandon_Rate,Returns,Segment_Label
0,CUST00001,51,13,1560,113,56,14.766242,33,828,1,28,17.629489,8,Silver
1,CUST00002,134,5,3,26,7,5.409044,2,60,0,4,23.024216,9,Iron
2,CUST00003,55,15,235,57,37,5.082231,11,710,1,17,28.391939,16,Copper
3,CUST00004,46,11,1293,191,48,18.936486,20,1223,0,17,15.192267,0,Silver
4,CUST00005,21,30,3602,239,90,26.461395,37,1892,0,57,11.427460,8,Gold


In [ ]:
df['event_time'] = pd.to_datetime(df['event_time'])
df['date'] = df['event_time'].dt.date


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer_ID           10000 non-null  object 
 1   Recency               10000 non-null  int64  
 2   Frequency             10000 non-null  int64  
 3   Monetary              10000 non-null  int64  
 4   Avg_Order_Value       10000 non-null  int64  
 5   Session_Count         10000 non-null  int64  
 6   Avg_Session_Duration  10000 non-null  float64
 7   Pages_Viewed          10000 non-null  int64  
 8   Clicks                10000 non-null  int64  
 9   Campaign_Response     10000 non-null  int64  
 10  Wishlist_Adds         10000 non-null  int64  
 11  Cart_Abandon_Rate     10000 non-null  float64
 12  Returns               10000 non-null  int64  
 13  Segment_Label         10000 non-null  object 
dtypes: float64(2), int64(10), object(2)
memory usage: 1.1+ MB


In [6]:
df.isnull().sum()


Customer_ID             0
Recency                 0
Frequency               0
Monetary                0
Avg_Order_Value         0
Session_Count           0
Avg_Session_Duration    0
Pages_Viewed            0
Clicks                  0
Campaign_Response       0
Wishlist_Adds           0
Cart_Abandon_Rate       0
Returns                 0
Segment_Label           0
dtype: int64

In [7]:
df = df.fillna(0)


In [8]:
def normalize(series):
    return 100 * (series - series.min()) / (series.max() - series.min())


In [13]:
#Engagement Risk (Low value=High risk)
df['engagement_risk'] = (
    normalize(df['Frequency']) +
    normalize(df['Session_Count']) +
    normalize(df['Avg_Session_Duration']) +
    normalize(df['Pages_Viewed']) +
    normalize(df['Clicks'])
) / 5


In [14]:
#Revenue Risk(Low revenue=Bad)
df['revenue_risk'] = (
    normalize(df['Monetary']) +
    normalize(df['Avg_Order_Value'])
) / 2


In [15]:
#Conversion Risk(High abandon = danger,Low wishlist = weak intent)

df['conversion_risk'] = (
    normalize(df['Cart_Abandon_Rate']) +
    normalize(df['Wishlist_Adds'])
) / 2


In [16]:
#Trust Risk
df['trust_risk'] = (
    normalize(df['Returns']) +
    normalize(df['Campaign_Response'])
) / 2


In [18]:
#Customer risk score 
df['customer_risk_score'] = (
    0.30 * df['engagement_risk'] +
    0.30 * df['revenue_risk'] +
    0.25 * df['conversion_risk'] +
    0.15 * df['trust_risk']
)


In [19]:
#Risk Level
def risk_label(score):
    if score <= 30:
        return "Low Risk"
    elif score <= 60:
        return "Medium Risk"
    else:
        return "High Risk"

df['risk_level'] = df['customer_risk_score'].apply(risk_label)


In [20]:
final_df = df[
    ['Customer_ID',
     'engagement_risk',
     'revenue_risk',
     'conversion_risk',
     'trust_risk',
     'customer_risk_score',
     'risk_level',
     'Segment_Label']
]


In [21]:
df.head()

,Customer_ID,Recency,Frequency,Monetary,Avg_Order_Value,Session_Count,Avg_Session_Duration,Pages_Viewed,Clicks,Campaign_Response,Wishlist_Adds,Cart_Abandon_Rate,Returns,Segment_Label,engagement_risk,revenue_risk,conversion_risk,trust_risk,customer_risk_score,risk_level
0,CUST00001,51,13,1560,113,56,14.766242,33,828,1,28,17.629489,8,Silver,24.989961,21.042977,31.768364,66.666667,31.751972,Medium Risk
1,CUST00002,134,5,3,26,7,5.409044,2,60,0,4,23.024216,9,Iron,4.219219,2.071563,25.043260,18.750000,10.960550,Low Risk
2,CUST00003,55,15,235,57,37,5.082231,11,710,1,17,28.391939,16,Copper,13.550939,7.216719,36.978014,83.333333,27.974801,Low Risk
3,CUST00004,46,11,1293,191,48,18.936486,20,1223,0,17,15.192267,0,Silver,24.864289,29.733016,23.774963,0.000000,22.322932,Low Risk
4,CUST00005,21,30,3602,239,90,26.461395,37,1892,0,57,11.427460,8,Gold,42.956682,47.453457,40.211213,16.666667,39.675845,Medium Risk


In [24]:
final_df.to_csv("C:/Users/akshi/OneDrive/Desktop/DA/project_3/customer_risk_scores.csv", index=False)
